In [41]:
import glob
import os
import re
import pandas as pd
import numpy as np

In [2]:
# helps sort file by number at the end of each filename (10 correctly placed after 9 in filename)
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [107]:
# concatenate all csv files
ejmr_data = sorted(glob.glob('ejmr_pg*.csv'), key=numericalSort)

df_list = []
for filename in sorted(ejmr_data):
    df_list.append(pd.read_csv(filename))
full_df = pd.concat(df_list)

In [108]:
# split title into orig. link and post title, get freshness values
full_df['post_link'] = full_df['title'].str.extract(r'<a href="(.*?)">',expand=True)
full_df['post_title'] = full_df['title'].str.extract(r'">(.*?)</a>',expand=True)
full_df['how_fresh'] = full_df['freshness'].str.extract(r'">(.*?)</a>',expand=True)

list(full_df.columns)

# reorder columns
full_df = full_df[['post_title',
                   'num_posts',
                   'views',
                   'votes',
                   'how_fresh',
                   'download_time',
                   'title',
                   'post_link',
                   'freshness'
                  ]]

# export full data to csv
full_df.to_csv('ejmr_data.csv', index=True, index_label='pandas_index')

In [109]:
full_df.head(n=25)

,post_title,num_posts,views,votes,how_fresh,download_time,title,post_link,freshness
0,Pls explain me how machines can learn?,5,95,0-0,1 day,2017-06-11 03:37:22.777761,"<a href=""https://www.econjobrumors.com/topic/p...",https://www.econjobrumors.com/topic/pls-explai...,"<a href=""https://www.econjobrumors.com/topic/p..."
1,Machine learning is Satan worshipping in disguise,17,337,6-0,1 day,2017-06-11 03:37:22.777852,"<a href=""https://www.econjobrumors.com/topic/m...",https://www.econjobrumors.com/topic/machine-le...,"<a href=""https://www.econjobrumors.com/topic/m..."
2,Should I buy a BMW,29,960,0-0,1 day,2017-06-11 03:37:22.777956,"<a href=""https://www.econjobrumors.com/topic/s...",https://www.econjobrumors.com/topic/should-i-b...,"<a href=""https://www.econjobrumors.com/topic/s..."
3,A proposal on how to replace student evaluations,22,561,4-3,1 day,2017-06-11 03:37:22.778058,"<a href=""https://www.econjobrumors.com/topic/a...",https://www.econjobrumors.com/topic/a-proposal...,"<a href=""https://www.econjobrumors.com/topic/a..."
4,"Roman Catholics, put down your rosaries and ex...",1,69,1-0,1 day,2017-06-11 03:37:22.778133,"<a href=""https://www.econjobrumors.com/topic/r...",https://www.econjobrumors.com/topic/roman-cath...,"<a href=""https://www.econjobrumors.com/topic/r..."
5,Eh...eh...brains....aARGHHH BRAINS,2,63,0-0,1 day,2017-06-11 03:37:22.778206,"<a href=""https://www.econjobrumors.com/topic/e...",https://www.econjobrumors.com/topic/ehehbrains...,"<a href=""https://www.econjobrumors.com/topic/e..."
6,Taylor Swift puts all music on Spotify on Katy...,7,365,1-1,1 day,2017-06-11 03:37:22.778279,"<a href=""https://www.econjobrumors.com/topic/t...",https://www.econjobrumors.com/topic/taylor-swi...,"<a href=""https://www.econjobrumors.com/topic/t..."
7,2016 Maryland Senior Hires,104,10160,0-0,1 day,2017-06-11 03:37:22.778472,"<a href=""https://www.econjobrumors.com/topic/2...",https://www.econjobrumors.com/topic/2016-maryl...,"<a href=""https://www.econjobrumors.com/topic/2..."
8,Seminar Notice from CCER,4,392,0-0,1 day,2017-06-11 03:37:22.778543,"<a href=""https://www.econjobrumors.com/topic/s...",https://www.econjobrumors.com/topic/seminar-no...,"<a href=""https://www.econjobrumors.com/topic/s..."
9,EJMR Rule #1: Be Respectful,71,783,3-7,1 day,2017-06-11 03:37:22.778686,"<a href=""https://www.econjobrumors.com/topic/e...",https://www.econjobrumors.com/topic/ejmr-rule-...,"<a href=""https://www.econjobrumors.com/topic/e..."


In [77]:
os.getcwd()
ejmr_df = pd.read_csv('ejmr_data/ejmr_data.csv')

In [78]:
ejmr_df.head(n=100)

,pandas_index,post_title,num_posts,views,votes,how_fresh,download_time,title,post_link,freshness
0,0,Pls explain me how machines can learn?,5,95,0-0,1 day,2017-06-11 03:37:22.777761,"<a href=""https://www.econjobrumors.com/topic/p...",https://www.econjobrumors.com/topic/pls-explai...,"<a href=""https://www.econjobrumors.com/topic/p..."
1,1,Machine learning is Satan worshipping in disguise,17,337,6-0,1 day,2017-06-11 03:37:22.777852,"<a href=""https://www.econjobrumors.com/topic/m...",https://www.econjobrumors.com/topic/machine-le...,"<a href=""https://www.econjobrumors.com/topic/m..."
2,2,Should I buy a BMW,29,960,0-0,1 day,2017-06-11 03:37:22.777956,"<a href=""https://www.econjobrumors.com/topic/s...",https://www.econjobrumors.com/topic/should-i-b...,"<a href=""https://www.econjobrumors.com/topic/s..."
3,3,A proposal on how to replace student evaluations,22,561,4-3,1 day,2017-06-11 03:37:22.778058,"<a href=""https://www.econjobrumors.com/topic/a...",https://www.econjobrumors.com/topic/a-proposal...,"<a href=""https://www.econjobrumors.com/topic/a..."
4,4,"Roman Catholics, put down your rosaries and ex...",1,69,1-0,1 day,2017-06-11 03:37:22.778133,"<a href=""https://www.econjobrumors.com/topic/r...",https://www.econjobrumors.com/topic/roman-cath...,"<a href=""https://www.econjobrumors.com/topic/r..."
5,5,Eh...eh...brains....aARGHHH BRAINS,2,63,0-0,1 day,2017-06-11 03:37:22.778206,"<a href=""https://www.econjobrumors.com/topic/e...",https://www.econjobrumors.com/topic/ehehbrains...,"<a href=""https://www.econjobrumors.com/topic/e..."
6,6,Taylor Swift puts all music on Spotify on Katy...,7,365,1-1,1 day,2017-06-11 03:37:22.778279,"<a href=""https://www.econjobrumors.com/topic/t...",https://www.econjobrumors.com/topic/taylor-swi...,"<a href=""https://www.econjobrumors.com/topic/t..."
7,7,2016 Maryland Senior Hires,104,10160,0-0,1 day,2017-06-11 03:37:22.778472,"<a href=""https://www.econjobrumors.com/topic/2...",https://www.econjobrumors.com/topic/2016-maryl...,"<a href=""https://www.econjobrumors.com/topic/2..."
8,8,Seminar Notice from CCER,4,392,0-0,1 day,2017-06-11 03:37:22.778543,"<a href=""https://www.econjobrumors.com/topic/s...",https://www.econjobrumors.com/topic/seminar-no...,"<a href=""https://www.econjobrumors.com/topic/s..."
9,9,EJMR Rule #1: Be Respectful,71,783,3-7,1 day,2017-06-11 03:37:22.778686,"<a href=""https://www.econjobrumors.com/topic/e...",https://www.econjobrumors.com/topic/ejmr-rule-...,"<a href=""https://www.econjobrumors.com/topic/e..."


In [79]:
def nonNaN_title_only(df, df_col):
    print "Total number of rows: {0}".format(len(df))
    print len(df) - df.count()  # number of NaN rows for each column
    
    df[df_col].replace('', np.nan, inplace=True)
    df.dropna(subset=[df_col], inplace=True)
    
    print "Total number of rows after deleting NaN: {0}".format(len(df))
    
    return df

def cleanText(text):
    """
    removes punctuation, stopwords and returns lowercase text in a list of single words
    source: http://francescopochetti.com/tag/sentiment-analysis/
    """
    
    print text
    
    if text: 
    
        text = text.lower()    

        from bs4 import BeautifulSoup
        text = BeautifulSoup(text).get_text()

        from nltk.tokenize import RegexpTokenizer
        tokenizer = RegexpTokenizer(r'\w+')
        text = tokenizer.tokenize(text)

        from nltk.corpus import stopwords
        clean = [word for word in text if word not in stopwords.words('english')]

        return clean

In [80]:
nonNaN_title_only(ejmr_df,'post_title')

Total number of rows: 288501
pandas_index        0
post_title       2385
num_posts           0
views               0
votes               0
how_fresh           0
download_time       0
title               0
post_link        2347
freshness           0
dtype: int64
Total number of rows after deleting NaN: 286116


,pandas_index,post_title,num_posts,views,votes,how_fresh,download_time,title,post_link,freshness
0,0,Pls explain me how machines can learn?,5,95,0-0,1 day,2017-06-11 03:37:22.777761,"<a href=""https://www.econjobrumors.com/topic/p...",https://www.econjobrumors.com/topic/pls-explai...,"<a href=""https://www.econjobrumors.com/topic/p..."
1,1,Machine learning is Satan worshipping in disguise,17,337,6-0,1 day,2017-06-11 03:37:22.777852,"<a href=""https://www.econjobrumors.com/topic/m...",https://www.econjobrumors.com/topic/machine-le...,"<a href=""https://www.econjobrumors.com/topic/m..."
2,2,Should I buy a BMW,29,960,0-0,1 day,2017-06-11 03:37:22.777956,"<a href=""https://www.econjobrumors.com/topic/s...",https://www.econjobrumors.com/topic/should-i-b...,"<a href=""https://www.econjobrumors.com/topic/s..."
3,3,A proposal on how to replace student evaluations,22,561,4-3,1 day,2017-06-11 03:37:22.778058,"<a href=""https://www.econjobrumors.com/topic/a...",https://www.econjobrumors.com/topic/a-proposal...,"<a href=""https://www.econjobrumors.com/topic/a..."
4,4,"Roman Catholics, put down your rosaries and ex...",1,69,1-0,1 day,2017-06-11 03:37:22.778133,"<a href=""https://www.econjobrumors.com/topic/r...",https://www.econjobrumors.com/topic/roman-cath...,"<a href=""https://www.econjobrumors.com/topic/r..."
5,5,Eh...eh...brains....aARGHHH BRAINS,2,63,0-0,1 day,2017-06-11 03:37:22.778206,"<a href=""https://www.econjobrumors.com/topic/e...",https://www.econjobrumors.com/topic/ehehbrains...,"<a href=""https://www.econjobrumors.com/topic/e..."
6,6,Taylor Swift puts all music on Spotify on Katy...,7,365,1-1,1 day,2017-06-11 03:37:22.778279,"<a href=""https://www.econjobrumors.com/topic/t...",https://www.econjobrumors.com/topic/taylor-swi...,"<a href=""https://www.econjobrumors.com/topic/t..."
7,7,2016 Maryland Senior Hires,104,10160,0-0,1 day,2017-06-11 03:37:22.778472,"<a href=""https://www.econjobrumors.com/topic/2...",https://www.econjobrumors.com/topic/2016-maryl...,"<a href=""https://www.econjobrumors.com/topic/2..."
8,8,Seminar Notice from CCER,4,392,0-0,1 day,2017-06-11 03:37:22.778543,"<a href=""https://www.econjobrumors.com/topic/s...",https://www.econjobrumors.com/topic/seminar-no...,"<a href=""https://www.econjobrumors.com/topic/s..."
9,9,EJMR Rule #1: Be Respectful,71,783,3-7,1 day,2017-06-11 03:37:22.778686,"<a href=""https://www.econjobrumors.com/topic/e...",https://www.econjobrumors.com/topic/ejmr-rule-...,"<a href=""https://www.econjobrumors.com/topic/e..."


In [69]:
print ejmr_df.count()
print len(ejmr_df)
print len(ejmr_df) - ejmr_df.count()

pandas_index     286116
post_title       286116
num_posts        286116
views            286116
votes            286116
how_fresh        286116
download_time    286116
title            286116
post_link        286116
freshness        286116
dtype: int64
286116
pandas_index     0
post_title       0
num_posts        0
views            0
votes            0
how_fresh        0
download_time    0
title            0
post_link        0
freshness        0
dtype: int64


In [38]:
print len(ejmr_df)
print ejmr_df['post_title'].count()

288501
286116


In [34]:
ejmr_df['cleanTitle'] = ejmr_df['post_title'].apply(cleanText)


s = r"String interpolation is a term used to describe the process of evaluating a string value that is contained as one or more placeholders. To put it simply, it helps developers with string formatting and concatenation."

ejmr_df.head()

Pls explain me how machines can learn?
Machine learning is Satan worshipping in disguise
Should I buy a BMW
A proposal on how to replace student evaluations
Roman Catholics, put down your rosaries and explain this...
Eh...eh...brains....aARGHHH BRAINS
Taylor Swift puts all music on Spotify on Katy Perry's release date
2016 Maryland Senior Hires
Seminar Notice from CCER
EJMR Rule #1: Be Respectful
No nudes, just cradle suplexes
Left academia
Current JFE editor participation rates
Phone App Ideas
Econ vs. Fen
My chat with Steve Davies on UK politics | MR
NDB - New Development Bank - Junior / Young Professionals
How is the THRED conference?
What is the share of math/stat PhD transfers do you have in your cohort?
Which states are moving to green energy most quickly? | MR
Why was the Qatar cut-off so extreme and sudden? | MR
Acceptable delays to resubmit paper at top tier finance journal?
The most important truth about hard work, and also reading, that  | MR
The show so far, a continuing se

AttributeError: 'float' object has no attribute 'lower'